In [ ]:
%load_ext autoreload
%autoreload 2
import tqdm, tqdm.notebook

tqdm.tqdm = tqdm.notebook.tqdm  # notebook-friendly progress bars
from pathlib import Path
import numpy as np

from hloc import (
    extract_features,
    match_features,
    reconstruction,
    visualization,
    pairs_from_exhaustive,
)
from hloc.visualization import plot_images, read_image
from hloc.utils import viz_3d
import torch

In [ ]:
torch.cuda.set_device(3)
print("torch.cuda.current_device()=", torch.cuda.get_device_name())

images = Path("/home/cv_stu_03/Hierarchical-Localization/project/images/YueyaBuilding/ReconstructionDataset")
sfm_dir= Path("/home/cv_stu_03/Hierarchical-Localization/project/output/YueyaBuilding/sfm")
sfm_pairs= Path("/home/cv_stu_03/Hierarchical-Localization/project/output/YueyaBuilding/pairs-sfm.txt")
loc_pairs = Path("/home/cv_stu_03/Hierarchical-Localization/project/output/YueyaBuilding/pairs-loc.txt")
features= Path("/home/cv_stu_03/Hierarchical-Localization/project/output/YueyaBuilding/features.h5")
matches= Path("/home/cv_stu_03/Hierarchical-Localization/project/output/YueyaBuilding/matches.h5")

# otuputs = Path("/home/cv_stu_03/Hierarchical-Localization/project/output/Library")
# references = [p.relative_to(images).as_posix() for p in (images ).iterdir()]

In [ ]:
# images = Path("images/MainLibrary")
# outputs = Path("outputs/MengminweiBuilding")
# # !rm -rf $outputs
# sfm_pairs = outputs / "pairs-sfm.txt"
# loc_pairs = outputs / "pairs-loc.txt"
# sfm_dir = outputs / "sfm"
# features = outputs / "features.h5"
# matches = outputs / "matches.h5"

feature_conf = extract_features.confs["disk"]
matcher_conf = match_features.confs["NN-superpoint"]

In [ ]:
references = [p.relative_to(images).as_posix() for p in (images ).iterdir()]
print(len(references), "mapping images")
plot_images([read_image(images / r) for r in references], dpi=25)

In [ ]:
# import torch

# torch.cuda.set_device(2)

extract_features.main(
    feature_conf, images, image_list=references, feature_path=features
)
pairs_from_exhaustive.main(sfm_pairs, image_list=references)
match_features.main(matcher_conf, sfm_pairs, features=features, matches=matches)

In [ ]:
model = reconstruction.main(
    sfm_dir, images, sfm_pairs, features, matches, image_list=references
)
fig = viz_3d.init_figure()
viz_3d.plot_reconstruction(
    fig, model, color="rgba(255,0,0,0.5)", name="mapping", points_rgb=True
)
fig.show()